In [1]:
import pandas as pd
import csv
import numpy as np
from keras.preprocessing.text import one_hot

Using TensorFlow backend.


In [2]:
def encoder(text):
    return np.array(list(map(lambda x: ord(x), list(text))))
def decoder(text):
    return list(map(lambda x: chr(x), list(text)))

In [3]:
with open('D:\\Sentron\\cornell movie-dialogs corpus\\movie_lines.txt') as movie_line_file:
    movie_lines = list(csv.reader(movie_line_file, delimiter='|'))[::-1]
    movie_conversations=[]
    conversation=[]
    previus_row_line=0
    a=0
    for row in movie_lines:
        a+=1
        if previus_row_line == 0:
            conversation.append(encoder(row[4]))
            previus_row_line=int(row[0][1:])
        elif int(row[0][1:])==previus_row_line+1:
            conversation.append(encoder(row[4]))
            previus_row_line=int(row[0][1:])
        else:
            movie_conversations.append(conversation)
            conversation=[encoder(row[4])]
            previus_row_line=int(row[0][1:])
    print(a)

303230


In [4]:
XY_conv=[]
X_pair1=[]
X_pair2=[]
Y_pair1=[]
Y_pair2=[]
for conv in movie_conversations:
    i=0
    X_pair1=[]
    X_pair2=[]
    Y_pair1=[]
    Y_pair2=[]
    for line in conv[:-1]:
        i+=1
        if i%2==1:
            X_pair1.append(line.reshape(1, line.shape[0]))
        else:
            X_pair2.append(line.reshape(1, line.shape[0]))
    i=0
    for line in conv[1:]:
        i+=1
        if i%2==1:
            Y_pair1.append(line.reshape(1, line.shape[0]))
        else:
            Y_pair2.append(line.reshape(1, line.shape[0]))
    if [X_pair1, Y_pair1]!=[[],[]]:
        XY_conv.append([X_pair1,Y_pair1])
    if [X_pair2, Y_pair2]!=[[],[]]:
        XY_conv.append([X_pair2,Y_pair2])

In [5]:
XY_conv[2][0][0].shape

(1, 56)

In [6]:
memory_size=(None,1)

In [7]:
from keras.layers import Input, LSTM, Dense, Concatenate, Reshape, Permute
from keras.models import Model
from keras.utils import plot_model
import keras
import pydotplus
from keras.utils.vis_utils import model_to_dot

memory_input=Input(shape=memory_size, name="memory")
text_input=Input(shape=([None, 1]), name="text")
concatenate_1=Concatenate(axis=1)([memory_input, text_input])
LSTM_1=LSTM(100, return_sequences=True, return_state=True)(concatenate_1)
LSTM_2=LSTM(100, return_sequences=True, return_state=True)(LSTM_1)
LSTM_3=LSTM(100, return_sequences=True)(LSTM_2)
Dense_1=Dense(100)(LSTM_3)
Dense_2=Dense(100)(Dense_1)
output_memory=Dense(1)(Dense_2)
output_text=Dense(1)(Dense_2)
model = Model(inputs=[memory_input,text_input], outputs=[output_memory,output_text], name="chat_bot")
plot_model(model, to_file='model.png', show_shapes=True)
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
memory (InputLayer)             (None, None, 1)      0                                            
__________________________________________________________________________________________________
text (InputLayer)               (None, None, 1)      0                                            
__________________________________________________________________________________________________
concatenate_1 (Concatenate)     (None, None, 1)      0           memory[0][0]                     
                                                                 text[0][0]                       
__________________________________________________________________________________________________
lstm_1 (LSTM)                   [(None, None, 100),  40800       concatenate_1[0][0]              
__________

In [8]:
from keras.optimizers import RMSprop
model.compile(loss='mean_squared_error', optimizer='adam', loss_weights=[0.0, 1.0])

In [9]:
train_conv = XY_conv

In [10]:
i=0
test_results=[]
for conv in train_conv:
    i+=1
    memory=np.zeros([1,10,1])
    for line in range(len(conv[0])):
        lineread0=np.expand_dims(conv[0][line], axis=3)
        lineread1=np.expand_dims(conv[1][line], axis=3)
        print(lineread0.shape)
        print(lineread1.shape)
        model.fit(x = [memory, lineread0],
                  y = [memory, lineread1])
        test_results.append(model.test([memory, np.expand_dims(conv[0][line], axis=3)],
                                       y=[memory, np.expand_dims(conv[1][line], axis=3)]))
        memory, textoutput = model.predict([memory, np.expand_dims(conv[0][line], axis=3)])

(1, 74, 1)
(1, 60, 1)


D:\Anaconda\lib\site-packages\ipykernel_launcher.py:7: DeprecationWarning: Both axis > a.ndim and axis < -a.ndim - 1 are deprecated and will raise an AxisError in the future.
  import sys
D:\Anaconda\lib\site-packages\ipykernel_launcher.py:8: DeprecationWarning: Both axis > a.ndim and axis < -a.ndim - 1 are deprecated and will raise an AxisError in the future.
  


Epoch 1/1


InvalidArgumentError: Incompatible shapes: [1,84,1] vs. [1,60,1]
	 [[Node: loss/dense_4_loss/sub = Sub[T=DT_FLOAT, _class=["loc:@training/Adam/gradients/loss/dense_4_loss/sub_grad/Reshape"], _device="/job:localhost/replica:0/task:0/device:CPU:0"](dense_4/add, _arg_dense_4_target_0_3)]]